In [3]:
import pandas as pd
import numpy as np
import requests
import ta
from ta.volume import VolumeWeightedAveragePrice
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator, StochasticOscillator, AwesomeOscillatorIndicator
from ta.trend import EMAIndicator, SMAIndicator, WMAIndicator, MACD
from scipy.stats import skew, kurtosis
from datetime import datetime
import warnings
import time
from concurrent.futures import ThreadPoolExecutor

warnings.filterwarnings('ignore')


def get_futures_symbol_list():

    url = 'https://fapi.binance.com/fapi/v1/exchangeInfo'
    data = requests.get(url).json()
    symbols = []
    for s in data['symbols']:
        if s['quoteAsset'] == 'USDT' and s['status'] == 'TRADING':
            if s['symbol'] not in ['KEYUSDT']:
                symbols.append(s['symbol'])
    return symbols


def fetch_futures_ohlcv(symbol, interval='15m', limit=500):

    url = f'https://fapi.binance.com/fapi/v1/klines?symbol={symbol}&interval={interval}&limit={limit}'
    try:
        data = requests.get(url, timeout=10).json()
        if 'code' in data:
            print(f"Error fetching data for {symbol}: {data['msg']}")
            return None
        df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume',
                                         'close_time', 'quote_asset_volume', 'number_of_trades',
                                         'taker_buy_base_volume', 'taker_buy_quote_volume', 'ignore'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]
        df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
        return df
    except Exception as e:
        print(f"An error occurred while fetching data for {symbol}: {e}")
        return None


def get_all_futures_data(symbols, interval='15m', limit=500):

    all_data = {}
    def fetch_and_store(symbol):
        print(f"Fetching data for {symbol}")
        df = fetch_futures_ohlcv(symbol, interval, limit)
        if df is not None and not df.empty:
            df.set_index('timestamp', inplace=True)
            all_data[symbol] = df

    with ThreadPoolExecutor(max_workers=10) as executor:
        executor.map(fetch_and_store, symbols)

    return all_data


def add_indicators_and_signals(all_data):

    for symbol, df in all_data.items():
        df['ema'] = EMAIndicator(close=df['close'], window=14).ema_indicator()
        df['sma'] = SMAIndicator(close=df['close'], window=14).sma_indicator()
        df['wma'] = WMAIndicator(close=df['close'], window=14).wma()
        df['vwma'] = (df['close'] * df['volume']).rolling(window=14).sum() / df['volume'].rolling(window=14).sum()
        df['rsi'] = RSIIndicator(close=df['close'], window=14).rsi()
        macd = MACD(close=df['close'])
        df['macd_diff'] = macd.macd_diff()
        df['stoch_rsi'] = ta.momentum.StochRSIIndicator(close=df['close'], window=14).stochrsi()
        df['obv'] = ta.volume.OnBalanceVolumeIndicator(close=df['close'], volume=df['volume']).on_balance_volume()
        high_low_avg = (df['high'] + df['low']) / 2
        df['fisher'] = np.arctanh(0.66 * ((high_low_avg - high_low_avg.rolling(5).min()) / (high_low_avg.rolling(5).max() - high_low_avg.rolling(5).min()) - 0.5))
        df['returns'] = df['close'].pct_change()
        df['variance'] = df['returns'].rolling(window=14).var()
        df['std_dev'] = df['returns'].rolling(window=14).std()
        df['skewness'] = df['returns'].rolling(window=14).apply(lambda x: skew(x.dropna()), raw=False)
        df['kurtosis'] = df['returns'].rolling(window=14).apply(lambda x: kurtosis(x.dropna()), raw=False)
        df['price_change'] = df['close'].diff()
        df['price_volume_corr'] = df['price_change'].rolling(window=14).corr(df['volume'])
        df['wt1'] = ta.momentum.WilliamsRIndicator(high=df['high'], low=df['low'], close=df['close'], lbp=9).williams_r()
        df['wt2'] = df['wt1'].rolling(window=3).mean()
        df['wt_cross'] = df['wt1'] - df['wt2']
        df['volume_change'] = df['volume'].pct_change()
        df['volume_increasing'] = df['volume_change'] > 0
        df['volume_increasing_consecutive'] = df['volume_increasing'].rolling(window=5).sum() == 5
        df['signal_strength'] = 0
        df['buy_signal'] = False
        df['sell_signal'] = False
        df['signal_strength'] += np.where(df['ema'] > df['sma'], 1, -1)
        df['signal_strength'] += np.where(df['sma'] > df['wma'], 1, -1)
        df['signal_strength'] += np.where(df['wma'] > df['vwma'], 1, -1)
        df['signal_strength'] += np.where(df['rsi'] < 40, 1, 0)
        df['signal_strength'] += np.where(df['rsi'] > 75, -1, 0)
        df['signal_strength'] += np.where(df['macd_diff'] > 0, 1, -1)
        df['signal_strength'] += np.where(df['stoch_rsi'] < 0.3, 1, 0)
        df['signal_strength'] += np.where(df['stoch_rsi'] > 0.8, -1, 0)
        df['signal_strength'] += np.where(df['obv'] > df['obv'].shift(1), 1, -1)
        df['signal_strength'] += np.where(df['fisher'] > df['fisher'].shift(1), 1, -1)
        df['signal_strength'] += np.where(df['variance'] < df['variance'].rolling(window=3).mean(), 1, 0)
        df['signal_strength'] += np.where(df['std_dev'] < df['std_dev'].rolling(window=3).mean(), 1, 0)
        df['signal_strength'] += np.where(df['skewness'] > 0, 1, -1)
        df['signal_strength'] += np.where(df['kurtosis'] > 3, 1, -1)
        df['signal_strength'] += np.where(df['price_volume_corr'] > 0, 1, -1)
        df['signal_strength'] += np.where(df['wt_cross'] > 0, 1, -1)
        df['signal_strength'] += np.where(df['volume_increasing_consecutive'], 1, 0)
        df['buy_signal'] = df['signal_strength'] >= 10
        df['sell_signal'] = df['signal_strength'] <= -10
        df.dropna(inplace=True)
    return all_data


def backtest(all_data, initial_balance=1000, risk_per_trade=0.01, leverage=5):
    balance = initial_balance
    in_position = False
    position_type = None 
    open_position = None
    entry_price = 0
    position_size = 0
    stop_loss_price = 0
    take_profit_price = 0
    trade_log = []
    commission_rate = 0.0002 
    target_profit_percentage = 0.03  
    stop_loss_percentage = 0.01  

    timestamps = all_data[next(iter(all_data))].index

    for time in timestamps:
        signals = []
        for symbol, df in all_data.items():
            if time in df.index:
                row = df.loc[time]
                if row['buy_signal']:
                    signals.append({'symbol': symbol, 'price': row['close'], 'action': 'buy'})
                elif row['sell_signal']:
                    signals.append({'symbol': symbol, 'price': row['close'], 'action': 'sell'})
        if in_position:
            if time in all_data[open_position].index:
                current_price = all_data[open_position].loc[time]['close']
                if position_type == 'long':
                    profit = (current_price - entry_price) * position_size
                elif position_type == 'short':
                    profit = (entry_price - current_price) * position_size
                if (position_type == 'long' and (current_price <= stop_loss_price or profit <= -max_loss)) or \
                   (position_type == 'short' and (current_price >= stop_loss_price or profit <= -max_loss)):
                    commission_exit = abs(position_size) * current_price * commission_rate
                    balance += profit - commission_exit
                    balance = max(balance, 0)
                    trade_log.append({'time': time, 'symbol': open_position, 'action': 'Exit', 'price': current_price,
                                      'profit': profit - commission_exit, 'balance': balance, 'reason': 'Stop-loss'})
                    print(f"Stop-loss hit for {open_position} ({position_type}) at {current_price:.4f} on {time}, Profit: {profit - commission_exit:.2f}, Balance: {balance:.2f}")
                    in_position = False
                    open_position = None
                elif (position_type == 'long' and current_price >= take_profit_price) or \
                     (position_type == 'short' and current_price <= take_profit_price):
                    commission_exit = abs(position_size) * current_price * commission_rate
                    balance += profit - commission_exit
                    balance = max(balance, 0)
                    trade_log.append({'time': time, 'symbol': open_position, 'action': 'Exit', 'price': current_price,
                                      'profit': profit - commission_exit, 'balance': balance, 'reason': 'Take-profit'})
                    print(f"Take-profit hit for {open_position} ({position_type}) at {current_price:.4f} on {time}, Profit: {profit - commission_exit:.2f}, Balance: {balance:.2f}")
                    in_position = False
                    open_position = None
        else:
            if signals:
                signal_strengths = []
                for signal in signals:
                    symbol = signal['symbol']
                    action = signal['action']
                    df = all_data[symbol]
                    row = df.loc[time]
                    signal_strengths.append({'symbol': symbol, 'action': action, 'price': signal['price'], 'signal_strength': row['signal_strength']})
                best_signal = max(signal_strengths, key=lambda x: abs(x['signal_strength']))
                symbol = best_signal['symbol']
                entry_price = best_signal['price']
                action = best_signal['action']
                max_loss = balance * risk_per_trade
                position_value = balance * leverage
                position_size = position_value / entry_price
                if action == 'buy':
                    position_type = 'long'
                    stop_loss_price = entry_price - (entry_price * stop_loss_percentage)
                    take_profit_price = entry_price + (entry_price * target_profit_percentage)
                elif action == 'sell':
                    position_type = 'short'
                    stop_loss_price = entry_price + (entry_price * stop_loss_percentage)
                    take_profit_price = entry_price - (entry_price * target_profit_percentage)
                commission_entry = position_value * commission_rate
                total_cost = commission_entry
                if total_cost > balance:
                    print(f"Not enough balance to open position for {symbol} at {entry_price:.4f} on {time}")
                    continue
                balance -= commission_entry
                open_position = symbol
                in_position = True
                trade_log.append({'time': time, 'symbol': symbol, 'action': action.capitalize(), 'price': entry_price,
                                  'position_size': position_size, 'balance': balance})
                print(f"{action.capitalize()}ing {symbol} at {entry_price:.4f} on {time}, Position Size: {position_size:.6f}, Balance: {balance:.2f}")

    if in_position:
        current_price = all_data[open_position].iloc[-1]['close']
        if position_type == 'long':
            profit = (current_price - entry_price) * position_size
        elif position_type == 'short':
            profit = (entry_price - current_price) * position_size
        commission_exit = abs(position_size) * current_price * commission_rate
        balance += profit - commission_exit
        balance = max(balance, 0)
        trade_log.append({'time': time, 'symbol': open_position, 'action': 'Exit', 'price': current_price,
                          'profit': profit - commission_exit, 'balance': balance, 'reason': 'End of data'})
        print(f"Closing position for {open_position} ({position_type}) at {current_price:.4f} on {time}, Profit: {profit - commission_exit:.2f}, Balance: {balance:.2f}")

    return balance, trade_log


def main():
    symbols = get_futures_symbol_list()
    symbols = symbols[:300] 

    interval = '15m'
    initial_balance = 1000
    risk_per_trade = 0.01
    leverage = 5

    start_time = time.time()

    all_data = get_all_futures_data(symbols, interval=interval, limit=500)
    all_data = add_indicators_and_signals(all_data)
    if not all_data:
        print("No data available after processing indicators.")
        return
    final_balance, trade_log = backtest(all_data, initial_balance, risk_per_trade, leverage)

    end_time = time.time()
    print(f"\nBacktest completed in {end_time - start_time:.2f} seconds.")

    print(f"\nInitial Balance: {initial_balance}")
    print(f"Final Balance: {final_balance:.2f}")
    profit_percent = ((final_balance - initial_balance) / initial_balance) * 100
    print(f"Total Profit: {profit_percent:.2f}%")

    trades_df = pd.DataFrame(trade_log)
    print("\nTrade Log:")
    print(trades_df)

if __name__ == "__main__":
    main()


Fetching data for BTCUSDT
Fetching data for ETHUSDT
Fetching data for BCHUSDT
Fetching data for XRPUSDT
Fetching data for EOSUSDT
Fetching data for LTCUSDT
Fetching data for TRXUSDT
Fetching data for ETCUSDT
Fetching data for LINKUSDT
Fetching data for XLMUSDT
Fetching data for ADAUSDT
Fetching data for XMRUSDT
Fetching data for DASHUSDT
Fetching data for ZECUSDT
Fetching data for XTZUSDT
Fetching data for BNBUSDT
Fetching data for ATOMUSDT
Fetching data for ONTUSDT
Fetching data for IOTAUSDT
Fetching data for BATUSDT
Fetching data for VETUSDT
Fetching data for NEOUSDT
Fetching data for QTUMUSDT
Fetching data for IOSTUSDT
Fetching data for THETAUSDT
Fetching data for ALGOUSDT
Fetching data for ZILUSDT
Fetching data for KNCUSDT
Fetching data for ZRXUSDT
Fetching data for COMPUSDT
Fetching data for OMGUSDT
Fetching data for DOGEUSDT
Fetching data for SXPUSDT
Fetching data for KAVAUSDT
Fetching data for BANDUSDT
Fetching data for RLCUSDT
Fetching data for MKRUSDT
Fetching data for SNXUSDT